# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

Загрузим необходимые библиотеки.

In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Импортируем датасет с данными, посмотрим на его первые 5 строк:

In [5]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Итак, в данном проекте задачей является закодирование таких признаков о клиенте, как `Пол`, `Возраст`, `Зарплата` и `Члены семьи`. Столбец `Страховые выплаты` является целевым признаком.

Создадим соответствующие датасеты с признаками `X` и целевыми признаками `y`.

In [6]:
X, y = data.drop('Страховые выплаты', axis=1), data['Страховые выплаты'] 

## Умножение матриц

В этом задании нам необходимо ответить на вопрос, поменяется ли качество модели линейной регрессии, если умножить признаки на обратимую матрицу.

Введем обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Нет, не поменяются.

**Обоснование:**

В формулу обучения вместо $X$ подставим скалярное произведение $X$ и $P$: $X P$

Тогда формула обучения $w'$ запишется в виде:

$$
w' = ((X P)^T (X P))^{-1} (X P)^T y
$$

Воспользуемся свойствами матриц: 

$$
(A B)^T = B^T A^T
$$

$$
(A B)^{-1} = B^{-1} A^{-1}
$$

$$
A A^{-1} = E
$$

$$
A A^{T} = E
$$

$$
(A^{T})^{-1} = (A^{-1})^T
$$

Теперь упростим нашу формулу $w'$, раскрыв скобки:

$$
w' = (P^T X^T X P)^{-1} P^T X^T y
$$

Раскроем скобки, отвечающие за операцию получения обратной матрицы:

$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

Сгруппируем соседние матрицы:

$$
w' = P^{-1} (X^T X)^{-1} (P^{-1})^T P^T X^T y
$$

$$
w' = P^{-1} (X^T X)^{-1} (P P^{-1})^T X^T y
$$

Получили единичную матрицу:

$$
w' = P^{-1} (X^T X)^{-1} E X^T y
$$

Любая матрица, умноженная на единичную, равна себе, поэтому ее можно сократить:

$$
w' = P^{-1} (X^T X)^{-1} X^T y
$$

Видно, что последние 4 матрицы и есть $w$:

$$
w' = P^{-1} w
$$

Подставим в формулу предсказаний линейной регрессии $a = Xw$ полученное значение $w'$ и $X P$:

$$
a' = X P P^{-1} w
$$

Получаем единичную матрицу:

$$
a' = X E w
$$

Сократив матрицу, получим:

$$
a' = X w = a
$$

Видно, что предсказания моделей будут одинаковы.

## Алгоритм преобразования

**Алгоритм**

Пусть нашими ключами будут две случайные матрицы $A$ и $B$, произведение которых дает обратимую матрицу $P$.
Напомним, что обратимой называется квадратная матрица, в которой определитель не равен нулю.

**Обоснование**

Если умножить матрицу $A$ размером $(N, M)$ и $B$ размером $(M, N)$, мы получим матрицу $P$ размером $(N, N)$, умножение на которую наших признаков не повлияет на модель линейной регрессии, что было доказано в предыдущем пункте. Значение N должно быть равным количеству строк в матрице признаков $X$.

**Создание матриц**

Создадим список с числами от 1 до 100.

In [7]:
numbers = list(range(1, 101))

Зададим размеры $N$ и $M$ наших рандомных матриц $A$ и $B$:

In [8]:
N = X.shape[1]
M = 42

Создадим рандомные матрицы:

In [9]:
A = np.array([np.random.choice(numbers, M, replace=True) for _ in range(N)])
B = np.array([np.random.choice(numbers, N, replace=True) for _ in range(M)])

Получим нашу обратную матрицу $P$:

In [10]:
P = np.dot(A, B)

Наконец, получим закодированную матрицу $X'$ путем умножения матриц $X$ и $P$:

In [11]:
X_P = np.dot(X, P)

Проверим размеры всех матриц:

In [12]:
print(f'''Размер матрицы с признаками: {X.shape}
Размер матрицы - ключа A: {A.shape}
Размер матрицы - ключа B: {B.shape}
Размер полученной квадратной обратимой матрицы P: {P.shape}
Размер полученной закодированной матрицы признаков X': {X_P.shape}
''')

Размер матрицы с признаками: (5000, 4)
Размер матрицы - ключа A: (4, 42)
Размер матрицы - ключа B: (42, 4)
Размер полученной квадратной обратимой матрицы P: (4, 4)
Размер полученной закодированной матрицы признаков X': (5000, 4)



## Проверка алгоритма

### Обучение модели с незакодированными признаками

Разобьем данные на тренировочную и тестовую выборки в соотношении 0.75/0.25.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Обучим модель.

In [14]:
clf_1 = LinearRegression()
clf_1.fit(X_train, y_train)

LinearRegression()

Посчитаем метрику качества модели - MSE на тренировочной выборке:

In [15]:
y_train_pred = clf_1.predict(X_train)
MSE_train = mean_squared_error(y_train, y_train_pred)
MSE_train

0.11832303990704156

Посчитаем метрику качества модели - MSE на тестовой выборке:

In [16]:
y_test_pred = clf_1.predict(X_test)
MSE_test = mean_squared_error(y_test, y_test_pred)
MSE_test

0.13862989572441428

**Вывод:**

Для модели с незакодированными признаками MSE на тренировочной и тестовой выборках составил 0.11832304 и 0.13862989, соответственно.

### Обучение модели с закодированными признаками

Разобьем данные на тренировочную и тестовую выборки в соотношении 0.75/0.25. Изменим параметр `random_state` для того, чтоб удостовериться, что модельс закодированными признаками работает на выборках, разбитых по-другому. 

In [17]:
X_P_train, X_P_test, y_P_train, y_P_test = train_test_split(X_P, y, test_size=0.25, random_state=42)

Обучим модель.

In [18]:
clf_2 = LinearRegression()
clf_2.fit(X_P_train, y_P_train)

LinearRegression()

Посчитаем метрику качества модели - MSE на тренировочной выборке:

In [19]:
y_P_train_pred = clf_2.predict(X_P_train)
MSE_P_train = mean_squared_error(y_P_train, y_P_train_pred)
MSE_P_train

0.118323039907025

Посчитаем метрику качества модели - MSE на тестовой выборке:

In [20]:
y_P_test_pred = clf_2.predict(X_P_test)
MSE_P_test = mean_squared_error(y_P_test, y_P_test_pred)
MSE_P_test

0.13862989572445894

**Вывод:**

Для модели с закодированными признаками MSE на тренировочной и тестовой выборках составил 0.11832304 и 0.13862989, соответственно.

## Общий вывод

В данном проекте были предприняты попытки закодирования матрицы признаков при обучении модели линейной регрессии для того, чтобы защитить данные о пользователях. Соответственно:
- На первом этапе было проведено ознакомление с данными. Был сделан вывод о том, что признаки о клиенте необходимо закодировать.
- На втором этапе было доказано, что при умножении матрицы признаков на обратимую матрицу на выходе будут получены те же предсказания в случае использования модели линейной регрессии.
- Затем был предложен алгоритм закодирования признаков, заключающийся в создании двух матриц-ключей $A$ и $B$, умножение которых дает обратимую матрицу $P$, на которую и умножается матрица с признаками $X$.
- Наконец, были проверены модели, использующие исходнеы и закодированные матрицы признаков. Соответствующие MSE для тестовых выборок оказались равны, что позволяет сделать вывод о том, что кодирование матрицы признаков не влияет на качество получаемой модели линейной регрессии.